In [10]:

import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler

In [11]:

wallet_df = pd.read_csv("Wallet id - Sheet1.csv")

In [12]:
wallet_df

,wallet_id
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3
1,0x06b51c6882b27cb05e712185531c1f74996dd988
2,0x0795732aacc448030ef374374eaae57d2965c16c
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae
...,...
98,0xf60304b534f74977e159b2e159e135475c245526
99,0xf67e8e5805835465f7eba988259db882ab726800
100,0xf7aa5d0752cfcd41b0a5945867d619a80c405e52
101,0xf80a8b9cfff0febf49914c269fb8aead4a22f847


In [13]:
# Simulate transaction and behavioral features

np.random.seed(42) 
n_wallets = len(wallet_df)

wallet_df["num_borrows"] = np.random.poisson(lam=5, size=n_wallets) 
wallet_df["total_borrowed"] = np.random.exponential(scale=10000, size=n_wallets)
wallet_df["repayment_ratio"] = np.clip(np.random.normal(loc=0.9, scale=0.1, size=n_wallets), 0, 1) 
wallet_df["num_liquidations"] = np.random.binomial(n=2, p=0.1, size=n_wallets) 
wallet_df["wallet_age_days"] = np.random.randint(30, 2000, size=n_wallets)
wallet_df["collateral_ratio"] = np.clip(np.random.normal(loc=1.5, scale=0.5, size=n_wallets), 1.0, 3.0)
wallet_df["tx_frequency"] = np.random.randint(5, 100, size=n_wallets)

In [14]:
# Feature engineering (inversion for risk modeling)
features = wallet_df[[
    "repayment_ratio",
    "num_liquidations",
    "wallet_age_days",
    "collateral_ratio",
    "tx_frequency"
]].copy()

features["inv_repayment_ratio"] = 1 - features["repayment_ratio"]
features["inv_wallet_age"] = features["wallet_age_days"].max() - features["wallet_age_days"]
features["inv_collateral_ratio"] = 3.0 - features["collateral_ratio"]
features["inv_tx_frequency"] = features["tx_frequency"].max() - features["tx_frequency"]

In [15]:
# Selecting risk-related features and normalize them
risk_features = features[[
    "inv_repayment_ratio",
    "num_liquidations",
    "inv_wallet_age",
    "inv_collateral_ratio",
    "inv_tx_frequency"
]]

scaler = MinMaxScaler()
normalized = scaler.fit_transform(risk_features)


In [16]:
# Computing final risk score (scale: 0–1000)
weights = np.array([0.3, 0.25, 0.15, 0.2, 0.1])
wallet_df["score"] = ((1 - normalized.dot(weights)) * 1000).round().astype(int)


In [17]:
# Saving results in CSV file named "wallet_scores.csv"
wallet_df[["wallet_id", "score"]].to_csv("wallet_scores.csv", index=False)
wallet_df[["wallet_id", "score"]].head()

,wallet_id,score
0,0x0039f22efb07a647557c7c5d17854cfd6d489ef3,491
1,0x06b51c6882b27cb05e712185531c1f74996dd988,721
2,0x0795732aacc448030ef374374eaae57d2965c16c,475
3,0x0aaa79f1a86bc8136cd0d1ca0d51964f4e3766f9,619
4,0x0fe383e5abc200055a7f391f94a5f5d1f844b9ae,586
